# Primer modelo automático usando H2O

## Importamos las librerías

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import h2o
from h2o.automl import H2OAutoML

## Importamos los datos con pandas

In [ ]:
# Cargando los datos
datos_titanic = #Carga los datos desde el archivo csv
entrenamiento, pruebas = #Separar los datos en entrenamiento y pruebas

In [ ]:
entrenamiento.describe()

In [ ]:
entrenamiento.head()

## Hacemos una "limpieza" de nuestro datos antes de hacer el modelo

In [ ]:
combine = [entrenamiento, pruebas]

In [ ]:
# Convertir valores en texto a números (0 y 1)
sex_mapping = {'male': 0, 'female': 1}
entrenamiento['Sex'] = entrenamiento['Sex'].map(sex_mapping)
pruebas['Sex'] = pruebas['Sex'].map(sex_mapping)

In [ ]:
calculo_edades = np.zeros((2,3))

In [ ]:
for dataset in combine:
    for sex in range(0, 2):
        for pclass in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == sex) & (dataset['Pclass'] == pclass+1)]['Age'].dropna()
            age_guess = guess_df.median()
            calculo_edades[sex, pclass] = int(age_guess/0.5 + 0.5) * 0.5
    
    for sex in range(0, 2):
        for pclass in range(0, 3):
            dataset.loc[(dataset.Age.isnull()) & (dataset.Sex == sex) &(dataset.Pclass == pclass+1),'Age'] = calculo_edades[sex, pclass]

In [ ]:
entrenamiento = entrenamiento.drop(['Ticket', 'Cabin', 'Name', 'PassengerId', 'SibSp', 'Parch', 'Embarked'], axis=1)
pruebas = pruebas.drop(['Ticket', 'Cabin', 'Name', 'PassengerId', 'SibSp', 'Parch', 'Embarked'], axis=1)

## Creamos y entrenamos nuestro modelo

In [ ]:
# Inicializar H2O

In [ ]:
entrenamiento = h2o.H2OFrame(entrenamiento)
pruebas = h2o.H2OFrame(pruebas)

In [ ]:
x = entrenamiento.columns
y = #seleccionar la clase que vamos a predecir
x.remove(y)

In [ ]:
entrenamiento[y] = entrenamiento[y].asfactor()

In [ ]:
aml_ti = #Usar la clase H2OAutoML con un tiempo máximo en segundos y una máxima cantidad de modelos

In [ ]:
aml_ti.train(x = x, y = y,
          training_frame = entrenamiento)

In [ ]:
lb_ti = aml_ti.leaderboard
lb_ti

In [ ]:
#Obtener detalles del modelo

## Predecimos con nuestro árbol

In [ ]:
Y_pred = aml_ti.leader.predict(pruebas)

In [ ]:
Y_pred